# Code for generating and running PH of noisy networks - dev

In [1]:
# Need to load in all the network generation functions here?


In [1]:
# Load packages

using Pkg
using Statistics
using Plots
using LinearAlgebra
using Distances
using Eirene
using StatsBase
using Random
using Distributions
using JLD
using MAT

include("graph_models.jl")
include("helper_functions.jl")


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-4197528488449458776\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-14114892437323299057\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-4197528488449458776\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

betticurveFromBarcode (generic function with 1 method)

## Choose weighed network and generate replicates

--------------------------------
## CHANGE PARAMETERS HERE
---------------------------------

In [2]:
## Pick from one of the graph models and generate out to THRESH edge density

nReps = 50
rho = 0.5      # Threshold at edge density = rho
nNodes = 70
save_figures = 1  # Boolean to save figure pdfs
save_data = 1    # Boolean to save data  

# for geometricConf
p = 0.01
scale_factor = 100

# for RG
dims = 3

# for discreteUniformConf
a = 0
b = 1000

# for cosineGeometric
dims = 3


# Graph model name one of "geometricConf", "IID" , "RG", "discreteUniformConf", "cosineGeometric", "RL", "assoc"
# "disassort", "coreperiph", "DP"
graph_model_names = ["geometricConf", "IID" , "RG", "discreteUniformConf", "cosineGeometric", "RL", "assoc","disassort", "coreperiph", "DP"]

10-element Array{String,1}:
 "geometricConf"      
 "IID"                
 "RG"                 
 "discreteUniformConf"
 "cosineGeometric"    
 "RL"                 
 "assoc"              
 "disassort"          
 "coreperiph"         
 "DP"                 

In [4]:

for graph_model_name in graph_model_names
    
    println(graph_model_name)
    
    # load in data
    
    # Extract only the noise (edges above the threshold)
    maxDim = 3
    nEdges = binomial(nNodes,2)
    barcode_array = Array{Array{Float64}}(undef,nReps,maxDim)
    betticurve_array = zeros(nEdges,maxDim,nReps)
    beginTime = time()



    for rep = 1:nReps

        G_i = weighted_graph_array_iidNoise_ord[:,:,rep]

        C = Eirene.eirene(G_i,model = "vr", maxdim = maxDim, record = "none")
        println("finished running eirene")
        for k in collect(1:maxDim)

            barcode_array[rep,k] = barcode(C,dim=k)

        end 

        #if rep%10 == 0
            printstyled("\nRun $(rep) finished.\n",color = :cyan)
        #end

        C=0
        endTime = time() - beginTime
        printstyled("Finished computing persistent homology, elapsed time $(endTime) seconds", color = :cyan)

    end



    endTime = time() - beginTime
    printstyled("Finished computing persistent homology, elapsed time $(endTime) seconds", color = :cyan)

    if save_data == 1
        save("../processed_data/$(betti_file_name)_eirene_output.jld",
            "barcode_array", barcode_array,
            "betticurve_array", betticurve_array)
    end


    printstyled("\nDone :)", color = :blue)


    beginTime = time()
    for rep = 1:nReps

        for k in collect(1:maxDim)

            betti_curve_i = betticurveFromBarcode(barcode_array[rep,k],nEdges)
            betticurve_array[:,k,rep] = betti_curve_i[1,:]
            

        end 


    end


    ## Plot average betti curves
    ### UPDATE FOR DIM 5

    if save_figures == 1
        betticurve_avg = dropdims(mean(betticurve_array,dims=3), dims = 3)
        betticurve_std = dropdims(std(betticurve_array,dims=3), dims = 3)

        x_axis = collect(1:nEdges) ./nEdges
        p1a = plot(x_axis,betticurve_avg[:,1],linewidth = 1.5, ribbon = betticurve_std[:,1], label = "Betti_1")
        for d = 2:maxDim
            plot!(x_axis,betticurve_avg[:,d],linewidth = 1.5, ribbon = betticurve_std[:,d], label = "Betti_$(d)")
        end

        title!("$(betti_file_name)")
        ylabel!("Betti_k")
        xlabel!("Edge Density")

        plot(p1a,layout = (1,1))


        savefig("../figures/$(betti_file_name)_bettis.pdf")
    end
end



geometricConf
One node left - try again
Thresholding at edge number 1208
One node left - try again
Thresholding at edge number 1208
One node left - try again
Thresholding at edge number 1208
One node left - try again
Thresholding at edge number 1208
One node left - try again
Thresholding at edge number 1208
One node left - try again
Thresholding at edge number 1208
One node left - try again
Thresholding at edge number 1208
One node left - try again
Thresholding at edge number 1208
One node left - try again
Thresholding at edge number 1208
One node left - try again
Thresholding at edge number 1208
One node left - try again
Thresholding at edge number 1208
One node left - try again
Thresholding at edge number 1208
One node left - try again
Thresholding at edge number 1208
One node left - try again
Thresholding at edge number 1208
One node left - try again
Thresholding at edge number 1208
One node left - try again
Thresholding at edge number 1208
One node left - try again
Thresholding at 

Finished computing persistent homology, elapsed time 130.58697295188904 secondsfinished running eirene

Run 34 finished.
Finished computing persistent homology, elapsed time 132.26025295257568 secondsfinished running eirene

Run 35 finished.
Finished computing persistent homology, elapsed time 138.47280406951904 secondsfinished running eirene

Run 36 finished.
Finished computing persistent homology, elapsed time 145.0984320640564 secondsfinished running eirene

Run 37 finished.
Finished computing persistent homology, elapsed time 148.3633451461792 secondsfinished running eirene

Run 38 finished.
Finished computing persistent homology, elapsed time 149.6693720817566 secondsfinished running eirene

Run 39 finished.
Finished computing persistent homology, elapsed time 151.06231999397278 secondsfinished running eirene

Run 40 finished.
Finished computing persistent homology, elapsed time 153.84636211395264 secondsfinished running eirene

Run 41 finished.
Finished computing persistent homol

Finished computing persistent homology, elapsed time 13.927119016647339 secondsfinished running eirene

Run 16 finished.
Finished computing persistent homology, elapsed time 14.837897062301636 secondsfinished running eirene

Run 17 finished.
Finished computing persistent homology, elapsed time 15.51576018333435 secondsfinished running eirene

Run 18 finished.
Finished computing persistent homology, elapsed time 16.41678214073181 secondsfinished running eirene

Run 19 finished.
Finished computing persistent homology, elapsed time 17.29324698448181 secondsfinished running eirene

Run 20 finished.
Finished computing persistent homology, elapsed time 18.20988416671753 secondsfinished running eirene

Run 21 finished.
Finished computing persistent homology, elapsed time 18.890743017196655 secondsfinished running eirene

Run 22 finished.
Finished computing persistent homology, elapsed time 19.74926209449768 secondsfinished running eirene

Run 23 finished.
Finished computing persistent homolog

Finished computing persistent homology, elapsed time 6.735230922698975 secondsfinished running eirene

Run 6 finished.
Finished computing persistent homology, elapsed time 8.184834003448486 secondsfinished running eirene

Run 7 finished.
Finished computing persistent homology, elapsed time 9.859426975250244 secondsfinished running eirene

Run 8 finished.
Finished computing persistent homology, elapsed time 11.046946048736572 secondsfinished running eirene

Run 9 finished.
Finished computing persistent homology, elapsed time 12.228227853775024 secondsfinished running eirene

Run 10 finished.
Finished computing persistent homology, elapsed time 13.91671895980835 secondsfinished running eirene

Run 11 finished.
Finished computing persistent homology, elapsed time 15.556807041168213 secondsfinished running eirene

Run 12 finished.
Finished computing persistent homology, elapsed time 16.664325952529907 secondsfinished running eirene

Run 13 finished.
Finished computing persistent homology, 

finished running eirene

Run 1 finished.
Finished computing persistent homology, elapsed time 1.4691200256347656 secondsfinished running eirene

Run 2 finished.
Finished computing persistent homology, elapsed time 3.682008981704712 secondsfinished running eirene

Run 3 finished.
Finished computing persistent homology, elapsed time 6.896018981933594 secondsfinished running eirene

Run 4 finished.
Finished computing persistent homology, elapsed time 10.055974960327148 secondsfinished running eirene

Run 5 finished.
Finished computing persistent homology, elapsed time 11.572489023208618 secondsfinished running eirene

Run 6 finished.
Finished computing persistent homology, elapsed time 13.25974988937378 secondsfinished running eirene

Run 7 finished.
Finished computing persistent homology, elapsed time 15.739490032196045 secondsfinished running eirene

Run 8 finished.
Finished computing persistent homology, elapsed time 17.94715404510498 secondsfinished running eirene

Run 9 finished.
Fin

Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Naming files disassort_700_100_100_20_20_09_05_05
finished running eirene

Run 1 finished.
Finished computing persistent homology, elapsed time 4.7155961990356445 secondsfinished running eirene

Run 2 finished.
Finished computing persistent homology, elapsed time 7.00651216506958 secondsfinished running eirene

Run 3 finished.
Finished computing persistent homology, elapsed time 9.898068189620972 secondsfinished running eirene

Run 4 finished.
Finished computing persistent homology, elapsed time 12.1

Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholding at edge number 1208
Thresholdi

Finished computing persistent homology, elapsed time 25.89370107650757 secondsfinished running eirene

Run 48 finished.
Finished computing persistent homology, elapsed time 26.459036111831665 secondsfinished running eirene

Run 49 finished.
Finished computing persistent homology, elapsed time 26.846349000930786 secondsfinished running eirene

Run 50 finished.
Finished computing persistent homology, elapsed time 27.561640977859497 secondsFinished computing persistent homology, elapsed time 27.56166410446167 seconds
Done :)